In [ ]:
!git clone https://github.com/ToelUl/Lattice-gauge-equivariant-CNN.git

!cp -r Lattice-gauge-equivariant-CNN/lge ./

In [ ]:
import numpy as np
import torch
import lge

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# U(1) group equivariant and invariant model

## Initialize the group

In [131]:
u1_group = lge.U1Group().to(device)
u1_rep_dim = u1_group.rep_dim
lie_algebra = u1_group.algebra
print(f"U(1) representation dimension: {u1_rep_dim}")
print(f"U(1) Lie algebra dimension: {lie_algebra.lie_alg_dim}")
print(f'Generator of U(1) Lie algebra : {lie_algebra.generators()}')
print(f'Identity element of U(1) group: {u1_group.identity}')

U(1) representation dimension: 1
U(1) Lie algebra dimension: 1
Generator of U(1) Lie algebra : [tensor(0.+1.j)]
Identity element of U(1) group: tensor([[1.+0.j]], device='cuda:0')


## Initialize the equivariant neural network

### 2D lattice

In [132]:
L = 4 # Lattice size
dims = [L, L] # Dimensions of the lattice
hidden_sizes = [5, 5] # Hidden layer sizes. Number of output channels (Wilson loops and links) per lattice site.
kernel_size = 2 # Kernel size or range for convolution, which means the maximum size of Wilson loop.
out_channels = len(dims) # Number of output channels (Wilson loops and links) per lattice site.

u1_gauge_equivariant_model_2d = lge.LgeConvNet(
    dims=dims,
    hidden_sizes=hidden_sizes,
    kernel_size=kernel_size,
    out_channels=out_channels,
    group=u1_group,
    gauge_invariant=False,
    use_norm=True,
    use_act_fn=True,
    threshold=1e-6
).to(device)

Check the model's structure

In [133]:
lge.check_model(u1_gauge_equivariant_model_2d)

LgeConvNet(
  (group): U1Group(
    (algebra): U1LieAlgebra()
  )
  (plaquette_layer): Plaquette(
    (group): U1Group(
      (algebra): U1LieAlgebra()
    )
  )
  (input_conv): LConvBilin()
  (after_input): Sequential(
    (0): LgeReLU()
    (1): TrNorm()
  )
  (hidden_block): Sequential(
    (0): LConvBilin()
    (1): LgeReLU()
    (2): TrNorm()
    (3): LConvBilin()
    (4): LgeReLU()
    (5): TrNorm()
  )
)
Total number of trainable parameters: 1184


### 4D lattice

In [134]:
L = 4 # Lattice size
dims = [L, L, L, L] # Dimensions of the lattice
hidden_sizes = [5, 5] # Hidden layer sizes. Number of output channels (Wilson loops and links) per lattice site.
kernel_size = 2 # Kernel size or range for convolution, which means the maximum size of Wilson loop.
out_channels = len(dims) # Number of output channels (Wilson loops and links) per lattice site.

u1_gauge_equivariant_model_4d = lge.LgeConvNet(
    dims=dims,
    hidden_sizes=hidden_sizes,
    kernel_size=kernel_size,
    out_channels=out_channels,
    group=u1_group,
    gauge_invariant=False,
    use_norm=True,
    use_act_fn=True,
    threshold=1e-6
).to(device)

Check the model's structure

In [135]:
lge.check_model(u1_gauge_equivariant_model_4d)

LgeConvNet(
  (group): U1Group(
    (algebra): U1LieAlgebra()
  )
  (plaquette_layer): Plaquette(
    (group): U1Group(
      (algebra): U1LieAlgebra()
    )
  )
  (input_conv): LConvBilin()
  (after_input): Sequential(
    (0): LgeReLU()
    (1): TrNorm()
  )
  (hidden_block): Sequential(
    (0): LConvBilin()
    (1): LgeReLU()
    (2): TrNorm()
    (3): LConvBilin()
    (4): LgeReLU()
    (5): TrNorm()
  )
)
Total number of trainable parameters: 1702


### Gauge invariant model with using the trace of the Wilson loop.

In [136]:
u1_gauge_invariant_model_4d = lge.LgeConvNet(
    dims=dims,
    hidden_sizes=hidden_sizes,
    kernel_size=kernel_size,
    out_channels=out_channels,
    group=u1_group,
    gauge_invariant=True,
    use_norm=True,
    use_act_fn=True,
    threshold=1e-6
).to(device)

Check the model's structure

In [137]:
lge.check_model(u1_gauge_invariant_model_4d)

LgeConvNet(
  (group): U1Group(
    (algebra): U1LieAlgebra()
  )
  (plaquette_layer): Plaquette(
    (group): U1Group(
      (algebra): U1LieAlgebra()
    )
  )
  (input_conv): LConvBilin()
  (after_input): Sequential(
    (0): LgeReLU()
    (1): TrNorm()
  )
  (hidden_block): Sequential(
    (0): LConvBilin()
    (1): LgeReLU()
    (2): TrNorm()
    (3): LConvBilin()
    (4): LgeReLU()
    (5): TrNorm()
    (6): LTrace()
  )
)
Total number of trainable parameters: 1702


### Baseline model

In [138]:
class BaselineModel(torch.nn.Module):
    def __init__(self, channels):
        super(BaselineModel, self).__init__()
        self.mlp1 = torch.nn.Linear(channels, 3)
        self.mlp2 = torch.nn.Linear(3, channels)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        shape = tuple(x.shape)
        x = torch.flatten(x, start_dim=1)
        x = self.mlp1(x)
        x = self.relu(x)
        x = self.mlp2(x)
        x = x.view(*shape)
        return x

baseline_model = BaselineModel(channels=np.prod(dims)*len(dims)*u1_rep_dim*u1_rep_dim*2).to(device)

lge.check_model(baseline_model)

BaselineModel(
  (mlp1): Linear(in_features=2048, out_features=3, bias=True)
  (mlp2): Linear(in_features=3, out_features=2048, bias=True)
  (relu): ReLU()
)
Total number of trainable parameters: 14339


## Initialize the gauge link

In [139]:
batch_size = 10
dim = len(dims) # Number of dimensions
num_spacial_points = np.prod(dims) # Number of spacial points
sample_size = batch_size * num_spacial_points * dim
link = u1_group.random_element(
    sample_size=sample_size,
    apply_map=True, # Apply the exponential map.
)
print(f"link shape: {link.shape}")
print(f'Check is unitary: {u1_group.is_unitary(link)}')

# Reshape the link to the desired shape. The shape should be (batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim).
link = link.view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
print(f"link shape: {link.shape}")
print(f'Check is unitary: {u1_group.is_unitary(link)}')

link shape: torch.Size([10240, 1, 1])
Check is unitary: True
link shape: torch.Size([10, 256, 4, 1, 1])
Check is unitary: True


Check the type of link

In [140]:
print(f'Link type: {link.type()}')

Link type: torch.cuda.ComplexFloatTensor


The input to the lattice gauge equivariant model should be a real tensor

In [141]:
link_real = u1_group.convert_to_real(link)
print(f'Link real type: {link_real.type()}')
print(f'Link real shape: {link_real.shape}')

Link real type: torch.cuda.FloatTensor
Link real shape: torch.Size([10, 256, 4, 1, 1, 2])


## Apply the gauge transformation

### Global gauge transformation

In [142]:
left_act = u1_group.left_action_on_Cn
right_act = u1_group.right_action_on_Cn

Choose a random element from the group

In [143]:
global_g = u1_group.random_element().to(device)
print(f'Random element shape: {global_g.shape}')
print(f'Check is unitary: {u1_group.is_unitary(global_g)}')

Random element shape: torch.Size([1, 1, 1])
Check is unitary: True


Left action of the group element on the link.
$g \cdot link$.

In [144]:
global_g_link = u1_group.left_action_on_Cn(global_g, link)
print(f'g_link shape: {global_g_link.shape}')

g_link shape: torch.Size([10, 256, 4, 1, 1])


Because the group is abelian, do the right action of the group element on the $g \cdot link$ should return the original link.

<br>$g \cdot link \cdot g^{-1} = link \cdot g \cdot g^{-1} = link$

In [145]:
global_g_link_g_inv = u1_group.right_action_on_Cn(global_g, global_g_link)
print(torch.allclose(link, global_g_link_g_inv))

True


And the way to apply the gauge transformation on gauge link is

<br> $g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu})$

<br>We can use the function `gauge_trans_to_gauge_link` to apply the gauge transformation on gauge link.

In [146]:
global_g_link_g_inv_ = lge.gauge_trans_to_gauge_link(
    u=link,
    global_group_element=global_g,
    local_group_elements=None,
    dims=None
)
# The output should be the same
print(torch.allclose(global_g_link_g_inv, global_g_link_g_inv_))

True


We can use the function `gauge_trans_to_wilson_loop` to apply the gauge transformation on closed loop.

In [147]:
w = lge.generate_wilson_loops(dims=dims, u=link, group=u1_group)
print(f'Wilson loops shape: {w.shape}')

global_g_w_g_inv = lge.gauge_trans_to_wilson_loop(
    w=w,
    global_group_element=global_g,
    local_group_elements=None
)
print(f'Global g*w*g_inv shape: {global_g_w_g_inv.shape}')

Wilson loops shape: torch.Size([10, 256, 6, 1, 1])
Global g*w*g_inv shape: torch.Size([10, 256, 6, 1, 1])


### Local gauge transformation

Choose random elements from the group for each spacial point.

In [148]:
spacial_points = batch_size * num_spacial_points
local_g = u1_group.random_element(sample_size=spacial_points, apply_map=True).to(device)
print(f'Random element shape: {local_g.shape}')
local_g = local_g.view(batch_size, num_spacial_points, 1, u1_rep_dim, u1_rep_dim)
print(f'Random element shape: {local_g.shape}')

Random element shape: torch.Size([2560, 1, 1])
Random element shape: torch.Size([10, 256, 1, 1, 1])


$g(x) \cdot W(x)$

In [149]:
w = lge.generate_wilson_loops(dims=dims, u=link, group=u1_group)
print(f'Wilson loops shape: {w.shape}')

local_g_w = left_act(local_g, w)
print(f'local_g_w shape: {local_g_w.shape}')

Wilson loops shape: torch.Size([10, 256, 6, 1, 1])
local_g_w shape: torch.Size([10, 256, 6, 1, 1])


Abelian group

<br>$g(x) \cdot W(x) \cdot g^{-1}(x) = W(x) \cdot g(x) \cdot g^{-1}(x) = W(x)$

In [150]:
local_g_w_g_inv = right_act(local_g, local_g_w)
print(torch.allclose(w, local_g_w_g_inv))

True


We can use the function `gauge_trans_to_wilson_loop` to apply the gauge transformation on closed loop.

In [151]:
local_g_w_g_inv_ = lge.gauge_trans_to_wilson_loop(
    w=w,
    global_group_element=None,
    local_group_elements=local_g,
)
print(f'Local g*w*g_inv shape: {local_g_w_g_inv.shape}')

# The output should be the same
print(torch.allclose(local_g_w_g_inv, local_g_w_g_inv_))

Local g*w*g_inv shape: torch.Size([10, 256, 6, 1, 1])
True


And the way to apply the gauge transformation on gauge link is

<br> $g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu})$

<br>We can use the function `gauge_trans_to_gauge_link` to apply the gauge transformation on gauge link.

In [152]:
local_g_link_g_inv = lge.gauge_trans_to_gauge_link(
    u=link,
    global_group_element=None,
    local_group_elements=local_g,
    dims=dims
)
print(f'Local g*link*g_inv shape: {local_g_link_g_inv.shape}')

Local g*link*g_inv shape: torch.Size([10, 256, 4, 1, 1])


### Gauge transformation with both global and local elements

In [153]:
g_w_g_inv = lge.gauge_trans_to_wilson_loop(
    w=w,
    global_group_element=global_g,
    local_group_elements=local_g,
)
print(f'g*w*g_inv shape: {g_w_g_inv.shape}')

g*w*g_inv shape: torch.Size([10, 256, 6, 1, 1])


In [154]:
g_link_g_inv = lge.gauge_trans_to_gauge_link(
    u=link,
    global_group_element=global_g,
    local_group_elements=local_g,
    dims=dims
)
print(f'g*link*g_inv shape: {g_link_g_inv.shape}')

g*link*g_inv shape: torch.Size([10, 256, 4, 1, 1])


## Check gauge equivariance and invariance

The group action on operators like the gauge link is defined as:

<br> $\mathcal{L}_{g(x), \mu} \cdot \hat{\mathcal{O}}_{\mu} \equiv g(x) \cdot \hat{\mathcal{O}}_{\mu} \cdot g^{-1}(x+\hat{\mu})$

<br>The group action on operators like the Wilson loop is defined as:

<br> $\mathcal{L}_{g(x)} \cdot W(x) \equiv g(x) \cdot W(x) \cdot g^{-1}(x)$

<br> The global gauge equivariance of the model is defined as:

<br> $\mathcal{L}_{g} \cdot Model\left(link_{\mu}\right) =  Model\left(\mathcal{L}_{g, \mu} \cdot link_{\mu}\right)$

<br> Where $g$ is a global group element.

<br> The global gauge invariance of the model is defined as:

<br> $Model\left(link_{\mu}\right) =  Model\left(\mathcal{L}_{g,\mu} \cdot link_{\mu}\right)$

<br>The local gauge equivariance of the model is defined as:

<br> $\mathcal{L}_{g(x)} \cdot Model\left(link(x)_{\mu}\right) =  Model\left(\mathcal{L}_{g(x),\mu} \cdot link(x)_{\mu}\right)$

<br> The local gauge invariance of the model is defined as:

<br> $Model\left(link(x)_{\mu}\right) =  Model\left(\mathcal{L}_{g(x),\mu} \cdot link(x)_{\mu}\right)$

### Check global gauge equivariance
#### (True for all network due to the abelian group)

The Wilson loop is always invariant under a global gauge transformation, and is always gauge equivalent when didn't trace the Wilson loop.

In [155]:
link = u1_group.random_element(sample_size=sample_size).view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
link_real = u1_group.convert_to_real(link)
global_g = u1_group.random_element().to(device)
global_g_link = u1_group.left_action_on_Cn(global_g, link)
global_g_link_g_inv = right_act(global_g, global_g_link)
print(f'Global g*link*g_inv shape: {global_g_link_g_inv.shape}')

Global g*link*g_inv shape: torch.Size([10, 256, 4, 1, 1])


Convert the link to real type

In [156]:
global_g_link_g_inv_real = u1_group.convert_to_real(global_g_link_g_inv)
print(f'Global g*link real type: {global_g_link_g_inv_real.type()}')
print(f'Global g*link real shape: {global_g_link_g_inv_real.shape}')

Global g*link real type: torch.cuda.FloatTensor
Global g*link real shape: torch.Size([10, 256, 4, 1, 1, 2])


Apply the model to the link and the transformed link.

<br>$Model \left(g \cdot link \cdot g^{-1} \right)$

<br>$Model \left( link \right)$

In [157]:
model_global_g_link_g_inv = u1_gauge_equivariant_model_4d(global_g_link_g_inv_real)
baseline_model_global_g_link_g_inv = baseline_model(global_g_link_g_inv_real)

model_link = u1_gauge_equivariant_model_4d(link_real)
baseline_model_link = baseline_model(link_real)

print('Lattice gauge equivariant model: ')
print(f'Model(g*link*g_inv) shape: {model_global_g_link_g_inv.shape}')
print(f'Model(link) shape: {model_link.shape}')
print('='*100)
print('Baseline model: ')
print(f'Model(g*link*g_inv) shape: {baseline_model_global_g_link_g_inv.shape}')
print(f'Model(link) shape: {baseline_model_link.shape}')

Lattice gauge equivariant model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1, 2])
Model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])
Baseline model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1, 2])
Model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])


Convert the output to complex type and compute

<br>$g \cdot Model(link) \cdot g^{-1}$

In [158]:
model_global_g_link_g_inv = u1_group.convert_to_complex(model_global_g_link_g_inv)
baseline_model_global_g_link_g_inv = u1_group.convert_to_complex(baseline_model_global_g_link_g_inv)

model_link = u1_group.convert_to_complex(model_link)
baseline_model_link = u1_group.convert_to_complex(baseline_model_link)

global_g_model_link_g_inv = right_act(global_g, left_act(global_g, model_link))
baseline_global_g_model_link_g_inv = right_act(global_g, left_act(global_g, baseline_model_link))

print('Lattice gauge equivariant model: ')
print(f'Model(g*link*g_inv) shape: {model_global_g_link_g_inv.shape}')
print(f'Model(link) shape: {model_link.shape}')
print('='*100)
print('Baseline model: ')
print(f'Model(g*link*g_inv) shape: {baseline_model_global_g_link_g_inv.shape}')
print(f'Model(link) shape: {baseline_model_link.shape}')

Lattice gauge equivariant model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1])
Model(link) shape: torch.Size([10, 256, 4, 1, 1])
Baseline model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1])
Model(link) shape: torch.Size([10, 256, 4, 1, 1])


#### Check the equivariance
<br> $g \cdot Model \left( link \right) \cdot g^{-1} = Model \left( g \cdot link \cdot g^{-1} \right)$

<br> But due to the numerical precision, the equality may not hold exactly.

<br> We can check the difference between the two outputs.

<br> $||g \cdot Model \left( link \right) \cdot g^{-1} - Model \left( g \cdot link \cdot g^{-1} \right)|| = \epsilon$

In [159]:
print('Lattice gauge equivariant model: ')
print(torch.allclose(global_g_model_link_g_inv, model_global_g_link_g_inv, atol=1e-3))
print('Error :', torch.norm(global_g_model_link_g_inv - model_global_g_link_g_inv).item())
print('='*100)
print('Baseline model: ')
print(torch.allclose(baseline_global_g_model_link_g_inv, baseline_model_global_g_link_g_inv, atol=1e-3))
print('Error :', torch.norm(baseline_global_g_model_link_g_inv - baseline_model_global_g_link_g_inv).item())

Lattice gauge equivariant model: 
True
Error : 2.102320286212489e-05
Baseline model: 
True
Error : 4.958920726494398e-06


### Check global gauge invariance
#### (True for all network due to the abelian group)

Apply the invariant model to the link and the transformed link.

<br>$Model \left(g \cdot link \cdot g^{-1} \right)$

<br>$Model \left( link \right)$

In [160]:
invariant_model_global_g_link_g_inv = u1_gauge_invariant_model_4d(global_g_link_g_inv_real)
baseline_global_g_model_link_g_inv = baseline_model(global_g_link_g_inv_real)

invariant_model_link = u1_gauge_invariant_model_4d(link_real)
baseline_model_link = baseline_model(link_real)

print('Lattice gauge invariant model: ')
print(f'Invariant model(g*link*g_inv) shape: {invariant_model_global_g_link_g_inv.shape}')
print(f'Invariant model(link) shape: {invariant_model_link.shape}')
print('='*100)
print('Baseline model: ')
print(f'Baseline model(g*link*g_inv) shape: {baseline_global_g_model_link_g_inv.shape}')
print(f'Baseline model(link) shape: {baseline_model_link.shape}')

Lattice gauge invariant model: 
Invariant model(g*link*g_inv) shape: torch.Size([10, 256, 4, 2])
Invariant model(link) shape: torch.Size([10, 256, 4, 2])
Baseline model: 
Baseline model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1, 2])
Baseline model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])


#### Check the invariance
<br> $Model \left( link \right) = Model \left( g \cdot link \cdot g^{-1} \right)$

<br> But due to the numerical precision, the equality may not hold exactly.

<br> We can check the difference between the two outputs.

<br> $||Model \left( link \right) - Model \left( g \cdot link \cdot g^{-1} \right)|| = \epsilon$

In [161]:
print('Lattice gauge invariant model: ')
print(torch.allclose(invariant_model_link, invariant_model_global_g_link_g_inv, atol=1e-3))
print('Error:', torch.norm(invariant_model_link - invariant_model_global_g_link_g_inv).item())
print('='*100)
print('Baseline model: ')
print(torch.allclose(baseline_model_link, baseline_global_g_model_link_g_inv, atol=1e-3))
print('Error:', torch.norm(baseline_model_link - baseline_global_g_model_link_g_inv).item())

Lattice gauge invariant model: 
True
Error: 4.223789346724516e-06
Baseline model: 
True
Error: 3.7628824429702945e-06


### Check local gauge equivariance

The Wilson loop is always invariant under a local gauge transformation, and is always local gauge equivalent when didn't trace the Wilson loop.

In [162]:
link = u1_group.random_element(sample_size=sample_size).view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)
link_real = u1_group.convert_to_real(link)
local_g = u1_group.random_element(sample_size=spacial_points).view(batch_size, num_spacial_points, 1, u1_rep_dim, u1_rep_dim).to(device)
local_g_link_g_inv = lge.gauge_trans_to_gauge_link(
    u=link,
    global_group_element=None,
    local_group_elements=local_g,
    dims=dims
)
print(f'Local g*link*g_inv shape: {local_g_link_g_inv.shape}')

Local g*link*g_inv shape: torch.Size([10, 256, 4, 1, 1])


Convert the tensor to real type for the model

In [163]:
local_g_link_g_inv_real = u1_group.convert_to_real(local_g_link_g_inv)
print(f'Local g*link*g_inv real shape: {local_g_link_g_inv_real.shape}')

Local g*link*g_inv real shape: torch.Size([10, 256, 4, 1, 1, 2])


Apply the model to the link and the transformed link.

<br>$Model \left( g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu}) \right)$

<br>$Model \left( link(x) \right)$

In [164]:
model_local_g_link_g_inv = u1_gauge_equivariant_model_4d(local_g_link_g_inv_real)
baseline_model_local_g_link_g_inv = baseline_model(local_g_link_g_inv_real)

model_link = u1_gauge_equivariant_model_4d(link_real)
baseline_model_link = baseline_model(link_real)

print('Lattice gauge equivariant model: ')
print(f'Model(g*link*g_inv) shape: {model_local_g_link_g_inv.shape}')
print(f'Model(link) shape: {model_link.shape}')
print('='*100)
print('Baseline model: ')
print(f'Model(g*link*g_inv) shape: {baseline_model_local_g_link_g_inv.shape}')
print(f'Model(link) shape: {baseline_model_link.shape}')

Lattice gauge equivariant model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1, 2])
Model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])
Baseline model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1, 2])
Model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])


Convert the output to complex type and compute

<br>$g(x) \cdot Model \left( link(x) \right) \cdot g^{-1}(x)$

In [165]:
model_local_g_link_g_inv = u1_group.convert_to_complex(model_local_g_link_g_inv)
baseline_model_local_g_link_g_inv = u1_group.convert_to_complex(baseline_model_local_g_link_g_inv)

model_link = u1_group.convert_to_complex(model_link)
baseline_model_link = u1_group.convert_to_complex(baseline_model_link)

local_g_model_link_g_inv = lge.gauge_trans_to_wilson_loop(
    w=model_link,
    global_group_element=None,
    local_group_elements=local_g,
)
baseline_local_g_model_link_g_inv = lge.gauge_trans_to_wilson_loop(
    w=baseline_model_link,
    global_group_element=None,
    local_group_elements=local_g,
)

print('Lattice gauge equivariant model: ')
print(f'Model(g*link*g_inv) shape: {model_local_g_link_g_inv.shape}')
print(f'Model(link) shape: {model_link.shape}')
print(f'Local g*Model(link)*g_inv shape: {local_g_model_link_g_inv.shape}')
print('='*100)
print('Baseline model: ')
print(f'Model(g*link*g_inv) shape: {baseline_model_local_g_link_g_inv.shape}')
print(f'Model(link) shape: {baseline_model_link.shape}')
print(f'Local g*Model(link)*g_inv shape: {baseline_local_g_model_link_g_inv.shape}')

Lattice gauge equivariant model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1])
Model(link) shape: torch.Size([10, 256, 4, 1, 1])
Local g*Model(link)*g_inv shape: torch.Size([10, 256, 4, 1, 1])
Baseline model: 
Model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1])
Model(link) shape: torch.Size([10, 256, 4, 1, 1])
Local g*Model(link)*g_inv shape: torch.Size([10, 256, 4, 1, 1])


#### Check the equivariance
<br> $g(x) \cdot Model \left( link(x)_{\mu} \right) \cdot g^{-1}(x+\hat{\mu}) = Model \left( g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu}) \right)$

<br> But due to the numerical precision, the equality may not hold exactly.

<br> We can check the difference between the two outputs.

<br> $||g(x) \cdot Model \left( link(x)_{\mu} \right) \cdot g^{-1}(x+\hat{\mu}) - Model \left( g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu}) \right)|| = \epsilon$

In [166]:
print('Lattice gauge equivariant model: ')
print(torch.allclose(local_g_model_link_g_inv, model_local_g_link_g_inv, atol=1e-3))
print('Error :', torch.norm(local_g_model_link_g_inv - model_local_g_link_g_inv).item())
print('='*100)
print('Baseline model: ')
print(torch.allclose(baseline_local_g_model_link_g_inv, baseline_model_local_g_link_g_inv, atol=1e-3))
print('Error :', torch.norm(baseline_local_g_model_link_g_inv - baseline_model_local_g_link_g_inv).item())

Lattice gauge equivariant model: 
True
Error : 1.9191054889233783e-05
Baseline model: 
False
Error : 26.964452743530273


### Local gauge invariance

Apply the invariant model to the link and the transformed link.

<br>$Model \left( g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu}) \right)$

<br>$Model \left( link(x)_{\mu} \right)$

In [167]:
invariant_model_local_g_link_g_inv = u1_gauge_invariant_model_4d(local_g_link_g_inv_real)
baseline_model_local_g_link_g_inv = baseline_model(local_g_link_g_inv_real)

invariant_model_link = u1_gauge_invariant_model_4d(link_real)
baseline_model_link = baseline_model(link_real)

print('Lattice gauge invariant model: ')
print(f'Invariant model(g*link*g_inv) shape: {invariant_model_local_g_link_g_inv.shape}')
print(f'Invariant model(link) shape: {invariant_model_link.shape}')
print('='*100)
print('Baseline model: ')
print(f'Baseline model(g*link*g_inv) shape: {baseline_model_local_g_link_g_inv.shape}')
print(f'Baseline model(link) shape: {baseline_model_link.shape}')

Lattice gauge invariant model: 
Invariant model(g*link*g_inv) shape: torch.Size([10, 256, 4, 2])
Invariant model(link) shape: torch.Size([10, 256, 4, 2])
Baseline model: 
Baseline model(g*link*g_inv) shape: torch.Size([10, 256, 4, 1, 1, 2])
Baseline model(link) shape: torch.Size([10, 256, 4, 1, 1, 2])


#### Check the invariance
<br> $Model \left( link(x) \right) = Model \left( g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu}) \right)$

<br> But due to the numerical precision, the equality may not hold exactly.

<br> We can check the difference between the two outputs.

<br> $||Model \left( link(x) \right) - Model \left( g(x) \cdot link(x)_{\mu} \cdot g^{-1}(x+\hat{\mu}) \right)|| = \epsilon$

In [168]:
print('Lattice gauge invariant model: ')
print(torch.allclose(invariant_model_link, invariant_model_local_g_link_g_inv, atol=1e-3))
print('Error:', torch.norm(invariant_model_link - invariant_model_local_g_link_g_inv).item())
print('='*100)
print('Baseline model: ')
print(torch.allclose(baseline_model_link, baseline_model_local_g_link_g_inv, atol=1e-3))
print('Error:', torch.norm(baseline_model_link - baseline_model_local_g_link_g_inv).item())

Lattice gauge invariant model: 
True
Error: 4.334547611506423e-06
Baseline model: 
False
Error: 26.964452743530273


## Check global phase invariance

<br> $Model \left( link \right) = Model \left( g \cdot link \right)$

In [169]:
link = u1_group.random_element(sample_size=sample_size).view(batch_size, num_spacial_points, dim, u1_rep_dim, u1_rep_dim)

global_g = u1_group.random_element().to(device)
global_g_link = u1_group.left_action_on_Cn(global_g, link)

global_g_link_real = u1_group.convert_to_real(global_g_link)
link_real = u1_group.convert_to_real(link)

invariant_model_link = u1_gauge_invariant_model_4d(link_real)
baseline_model_link = baseline_model(link_real)

invariant_model_global_g_link = u1_gauge_invariant_model_4d(global_g_link_real)
baseline_model_global_g_link = baseline_model(global_g_link_real)

print('Lattice gauge invariant model: ')
print(torch.allclose(invariant_model_link, invariant_model_global_g_link, atol=1e-3))
print('Error:', torch.norm(invariant_model_link - invariant_model_global_g_link).item())
print('='*100)
print('Baseline model: ')
print(torch.allclose(baseline_model_link, baseline_model_global_g_link, atol=1e-3))
print('Error:', torch.norm(baseline_model_link - baseline_model_global_g_link).item())

Lattice gauge invariant model: 
True
Error: 3.1458041576115647e-06
Baseline model: 
False
Error: 29.229198455810547
